<h3><b>Installing and importing necessary libraries</b></h3>

In [2]:
!pip install requests
!pip install apimoex
!pip install pandas

In [2]:
import requests
import apimoex
from datetime import datetime
import pandas as pd
import pathlib
import xml.etree.ElementTree as ET

<h3><b>Parsing of candle data for stocks and indexes</b></h3>

In [18]:
# opening the txt files with the ticker names of necessary companies
with open("TICK_DIV.txt", "r") as TICKs_file:
    TICKs = [line.rstrip() for line in TICKs_file]

with open("TICK_IND.txt", "r") as TICK2s_file:
    TICK2s = [line.rstrip() for line in TICK2s_file]

pathlib.Path("Database").mkdir(parents=True, exist_ok=True)

# empty lists to collect DataFrames
all_data_frames = []
all_data_frames_index = []

# additional info for parsing
end_date = datetime.now()
day = end_date.day
month = end_date.month
process = 0
process2 = 0

# parsing process of candles info
with requests.Session() as session:
    # parsing process of candles info (for stocks)
    for i in range(0, len(TICKs)):
        process += 1
        print(f"Parsing candle data for the necessary tickers {(process / len(TICKs)) * 100} %")

        # parsing stocks data for the ticker
        data = apimoex.get_market_candles(session, TICKs[i], start=f"2020-{month}-{day}", end=f"2025-{month}-{day}")
        if not data:  # skiping if there is no data for the ticker
            continue

        # creating a DataFrame from the candles data (of stocks)
        df = pd.DataFrame(data)
        # adding a new column for the STOCK_TICK and period values
        df['STOCK_TICK'] = TICKs[i]
        df['period'] = 'D'

        # appending the DataFrame to the list
        all_data_frames.append(df)
    
    # parsing process of candles info (for indexes)
    for i in range(0, len(TICK2s)):
        process2 += 1
        print(f"Parsing index data for the necessary tickers {(process2 / len(TICK2s)) * 100} %")

        # parsing indexes data for the ticker
        data_index = apimoex.get_market_candles(session, TICK2s[i], start=f"2020-{month}-{day}", end=f"2025-{month}-{day}", market="index")
        if not data_index:  # skiping if there is no data for the ticker
            continue
        
        # parsing of data about what's inside an index (which tickers)
        p3 = 0
        for di in data_index:
            p3 += 1
            print(f'For {TICK2s[i]} ticker parsing. Currently parsed: {p3/len(data_index) * 100} %')
            di['tickers'] = set()
            data_index_tick = apimoex.get_index_tickers(session, TICK2s[i], di['begin'])
            for dit in data_index_tick:
                di['tickers'].add(dit['ticker'])

        # crearing a DataFrame from the candles data (of indexes)
        df_index = pd.DataFrame(data_index)
        # adding a new column for the INDEX_TICK and perion values
        df_index['INDEX_TICK'] = TICK2s[i]
        df_index['period'] = 'D'
        # dropping unnecessary columns
        df_index = df_index.drop(columns=['volume'])
        # appending the DataFrame to the list
        all_data_frames_index.append(df_index)

# concatenating all the collected DataFrames into one (for stocks info)
combined_data = pd.concat(all_data_frames, ignore_index=True) 
# saving the combined DataFrame to a single JSON file (for stocks info)
combined_data.to_json("Database/combined_data_stock.json", orient='records', lines=True)

# concatenating all the collected DataFrames into one (for indexes info)
combined_data_index = pd.concat(all_data_frames_index, ignore_index=True)
# Save the combined DataFrame to a single JSON file (for indexes info)
combined_data_index.to_json("Database/combined_data_index.json", orient='records', lines=True)

Parsing candle data for the necessary tickers 2.631578947368421 %
Parsing candle data for the necessary tickers 5.263157894736842 %
Parsing candle data for the necessary tickers 7.894736842105263 %
Parsing candle data for the necessary tickers 10.526315789473683 %
Parsing candle data for the necessary tickers 13.157894736842104 %
Parsing candle data for the necessary tickers 15.789473684210526 %
Parsing candle data for the necessary tickers 18.421052631578945 %
Parsing candle data for the necessary tickers 21.052631578947366 %
Parsing candle data for the necessary tickers 23.684210526315788 %
Parsing candle data for the necessary tickers 26.31578947368421 %
Parsing candle data for the necessary tickers 28.947368421052634 %
Parsing candle data for the necessary tickers 31.57894736842105 %
Parsing candle data for the necessary tickers 34.21052631578947 %
Parsing candle data for the necessary tickers 36.84210526315789 %
Parsing candle data for the necessary tickers 39.473684210526315 %
Pa

In [71]:
# resampling to monthly frequency stocks data
monthly_df_list_stocks = []
for ticker in TICKs:
    # filtering data for the current ticker
    ticker_data = combined_data[combined_data['STOCK_TICK'] == ticker]
  
    # setting 'begin' as datetime index for resampling
    ticker_data['begin'] = pd.to_datetime(ticker_data['begin'])
    ticker_data.set_index('begin', inplace=True)
    
    # resampling to monthly frequency
    monthly_df = ticker_data.resample('ME').agg({
        'open': 'first',
        'close': 'last', 
        'high': 'max',
        'low': 'min',
        'value': 'sum',
        'volume': 'sum',
        'STOCK_TICK': lambda x: ticker,
        'period': lambda x: 'M'
    })
    monthly_df.reset_index(inplace=True)
    monthly_df_list_stocks.append(monthly_df)

# combining all monthly data
fin_stock_data = pd.concat(monthly_df_list_stocks)

# combining with original daily data
combined_data['begin'] = combined_data['begin'].astype(str)
fin_stock_data['begin'] = fin_stock_data['begin'].astype(str)
final_combined_data = pd.concat([combined_data, fin_stock_data])
final_combined_data.to_json("Database/combined_data_stock.json", orient='records', lines=True)
final_combined_data

/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_5594/811505522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['begin'] = pd.to_datetime(ticker_data['begin'])
/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_5594/811505522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['begin'] = pd.to_datetime(ticker_data['begin'])
/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_5594/811505522.py:8: SettingWithCopyWarning: 
A value is trying to be set on a co

,begin,open,close,high,low,value,volume,STOCK_TICK,period
0,2020-03-31,389.9,390.00,390.00,386.3,6.398647e+07,164340,MVID,D
1,2020-04-01,388.3,385.00,389.90,385.0,2.024915e+07,52170,MVID,D
2,2020-04-02,389.3,371.50,389.90,368.6,9.524656e+07,252420,MVID,D
3,2020-04-03,369.9,365.10,373.80,365.1,5.497840e+07,149160,MVID,D
4,2020-04-06,368.7,366.90,370.80,365.0,4.776447e+07,130190,MVID,D
...,...,...,...,...,...,...,...,...,...
56,2024-11-30,1473.0,1412.45,1548.95,1390.0,6.113613e+10,4201198,PLZL,M
57,2024-12-31,1412.5,1398.10,1515.90,1305.0,5.650342e+10,4024599,PLZL,M
58,2025-01-31,1405.0,1717.00,1743.80,1398.8,5.846154e+10,3639624,PLZL,M
59,2025-02-28,1713.0,1931.30,1985.35,1655.2,8.046951e+10,4329115,PLZL,M


In [88]:
# resampling to monthly frequency indexes data
monthly_df_list_indexes = []
for ticker in TICK2s:
    # filtering data for the current ticker
    ticker_data = combined_data_index[combined_data_index['INDEX_TICK'] == ticker]
    if isinstance(ticker_data.index, pd.DatetimeIndex):
        ticker_data.reset_index(inplace=True)  # moving date index to column 'begin'
    
    # setting 'begin' as datetime index for resampling
    ticker_data['begin'] = pd.to_datetime(ticker_data['begin'])
    ticker_data.set_index('begin', inplace=True)
    
    # resampling to monthly frequency
    monthly_df = ticker_data.resample('ME').agg({
        'open': 'first',
        'close': 'last', 
        'high': 'max',
        'low': 'min',
        'value': 'sum',
        'tickers': 'first', 
        'INDEX_TICK': lambda x: ticker,
        'period': lambda x: 'M'
    })
    monthly_df.reset_index(inplace=True)
    monthly_df_list_indexes.append(monthly_df)

# combining all monthly data
fin_index_data = pd.concat(monthly_df_list_indexes)

# combining with original daily data
combined_data_index['begin'] = combined_data_index['begin'].astype(str)
fin_index_data['begin'] = fin_index_data['begin'].astype(str)
final_combined_data_index = pd.concat([combined_data_index, fin_index_data])
final_combined_data_index.to_json("Database/combined_data_index.json", orient='records', lines=True)
final_combined_data_index

/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_5594/2730218808.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['begin'] = pd.to_datetime(ticker_data['begin'])
/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_5594/2730218808.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_data['begin'] = pd.to_datetime(ticker_data['begin'])
/var/folders/m0/9dr3jfm12bj1fkg2xjqh62jc0000gn/T/ipykernel_5594/2730218808.py:10: SettingWithCopyWarning: 
A value is trying to be set o

,begin,open,close,high,low,value,tickers,INDEX_TICK,period
0,2020-03-31,2466.28,2508.81,2526.13,2459.19,1.137560e+11,"{PLZL, LKOH, CBOM, IRAO, HYDR, ROSN, DSKY, UPR...",IMOEX,D
1,2020-04-01,2465.56,2473.61,2491.66,2449.01,8.025614e+10,"{PLZL, LKOH, CBOM, IRAO, HYDR, ROSN, DSKY, UPR...",IMOEX,D
2,2020-04-02,2528.72,2545.95,2562.54,2470.85,1.342401e+11,"{PLZL, LKOH, CBOM, IRAO, HYDR, ROSN, DSKY, UPR...",IMOEX,D
3,2020-04-03,2547.15,2572.23,2609.16,2533.06,1.171090e+11,"{PLZL, LKOH, CBOM, IRAO, HYDR, ROSN, DSKY, UPR...",IMOEX,D
4,2020-04-06,2597.28,2622.59,2639.56,2576.28,8.982314e+10,"{PLZL, LKOH, CBOM, IRAO, HYDR, ROSN, DSKY, UPR...",IMOEX,D
...,...,...,...,...,...,...,...,...,...
56,2024-11-30,8474.52,8707.29,9431.48,8198.45,7.276725e+11,"{SBER, RENI, VTBR, LEAS, BSPB, ZAYM, CBOM, SBE...",MOEXFN,M
57,2024-12-31,8691.27,9980.35,9980.35,8217.58,8.630320e+11,"{SBER, RENI, LEAS, BSPB, ZAYM, CBOM, SBERP, MO...",MOEXFN,M
58,2025-01-31,10054.27,10250.10,10500.66,9691.62,6.314465e+11,"{SBER, RENI, SPBE, LEAS, BSPB, ZAYM, CBOM, SBE...",MOEXFN,M
59,2025-02-28,10207.07,11177.56,11726.61,9972.30,7.774956e+11,"{SBER, RENI, SPBE, LEAS, BSPB, ZAYM, CBOM, SBE...",MOEXFN,M


<h3><b>Parsing of dividends data for stocks</b></h3>

In [19]:
# the URLs for the API endpoint
url1 = "https://iss.moex.com/iss/securities/"
url2 = "/dividends.xml"

# opening the txt file with the ticker names of necessary companies
with open("TICK_DIV.txt", "r") as TICK3s_file:
    TICK3s = [line.rstrip() for line in TICK3s_file]

# empty list to collect DataFrames
div_data = []
process3 = 0

# parsing process of dividends data
for i in range(len(TICK3s)):
    process3 += 1
    print(f"Parsing dividends data for the necessary tickers {(process3 / len(TICK3s)) * 100} %")
    response = requests.get(url1 + TICK3s[i] + url2)

    # parsing in case of success
    if response.status_code == 200:
        xml_data = response.content
        xml_string = xml_data.decode('utf-8')
        root = ET.fromstring(xml_string)

        for row in root.findall('.//row'):
            registry_closed_date = row.attrib.get('registryclosedate')
            # constraints on the necessary data format/values
            if registry_closed_date and int(registry_closed_date[0:4]) >= 2019 and int(registry_closed_date[0:4]) <= 2025:
                # a tuple to represent the unique entry
                unique_entry = row.attrib
                # appending the DataFrame to the list
                div_data.append(unique_entry)
                    
    # in case of fail
    else:
        print(f"Failed to retrieve data, status code: {response.status_code}")

# converting to DataFrame
df = pd.DataFrame(div_data)
df.rename(columns={'secid': 'DIV_TICK'}, inplace=True)

# converting the DataFrame to a JSON string
json_data = df.to_json("Database/combined_data_dividends.json", orient='records', lines=True)

Parsing dividends data for the necessary tickers 2.631578947368421 %
Parsing dividends data for the necessary tickers 5.263157894736842 %
Parsing dividends data for the necessary tickers 7.894736842105263 %
Parsing dividends data for the necessary tickers 10.526315789473683 %
Parsing dividends data for the necessary tickers 13.157894736842104 %
Parsing dividends data for the necessary tickers 15.789473684210526 %
Parsing dividends data for the necessary tickers 18.421052631578945 %
Parsing dividends data for the necessary tickers 21.052631578947366 %
Parsing dividends data for the necessary tickers 23.684210526315788 %
Parsing dividends data for the necessary tickers 26.31578947368421 %
Parsing dividends data for the necessary tickers 28.947368421052634 %
Parsing dividends data for the necessary tickers 31.57894736842105 %
Parsing dividends data for the necessary tickers 34.21052631578947 %
Parsing dividends data for the necessary tickers 36.84210526315789 %
Parsing dividends data for 